<a href="https://colab.research.google.com/github/a-memme/competitor_analytics_scrape/blob/main/Copy_Snapchat_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Dependencies**

In [ ]:
#Update
! apt-get update
#Temporarily install firefox
! apt install firefox  xvfb > /dev/null
#Selenium
! pip3 install  pyvirtualdisplay selenium webdriver_manager > /dev/null

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,347 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,372 kB]
Get:14 h

In [ ]:
#Import Dependencies
import time
import numpy as np
import re
import pandas as pd
from collections import OrderedDict

from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup

from selenium.webdriver.firefox.service import Service

from selenium.webdriver.firefox.options import Options

from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver import FirefoxOptions, DesiredCapabilities
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile

# selenium 4
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common import exceptions
from selenium.webdriver.common.proxy import Proxy, ProxyType


pd.set_option('display.max_colwidth', None)

# **Define Functions**

In [ ]:
#Define All Functions

#Driver configuration function
def driver_no_proxy():
  opts = Options()
  opts.add_argument("--headless")

  #Disable caching, memory, cookies, etc
  opts.set_preference('browser.cache.disk.enable', False)
  opts.set_preference('browser.cache.memory.enable', False)
  opts.set_preference('browser.cache.offline.enable', False)
  opts.set_preference('network.cookie.cookieBehavior', 2)

  service = Service(executable_path=GeckoDriverManager().install())
  browser = webdriver.Firefox(service=service, options=opts)
  return browser


#Function for identifying proxy ip
def proxy_identification():
  #Configure browser
  browser = driver_no_proxy()

  #Go to proxy testing webiste
  proxy_test = 'https://www.lagado.com/tools/proxy-test'

  browser.get(proxy_test)

  #Isolate the ip address from the proxy test results and convert into string
  ip_string = browser.find_element(By.XPATH, '//*[@id="summary"]/p[4]').text
  add_list = re.findall('\d*\.\d*\.\d*\.\d*', ip_string)
  ip_address = "".join(add_list)
  return ip_address


#Function for sfy feed list
def sfy_scrape():
  #Call driver set-up function to configure browser
  browser = driver_no_proxy()

  #Visit snap discover webpage
  time.sleep(10)
  browser.delete_all_cookies()
  snap_discover = 'https://www.snapchat.com/discover'
  browser.get(snap_discover)

  #SFY list for the Snapchat Scrape
  #Get all available links in SFY feed before scrolling
  all_links = []
  elems = browser.find_elements(By.CSS_SELECTOR, ".SuggestionsFeed_profileCardWrapper__cLvTI [href]")
  links = [elem.get_attribute('href') for elem in elems]

  all_links.append(links)

  #Create a loop where browser scrolls down to load new SFY suggestions and store links to channels in a list
  for x in range(20):
    time.sleep(1)
    browser.execute_script("window.scrollBy(0,6000)","")
    elems = browser.find_elements(By.CSS_SELECTOR, ".SuggestionsFeed_profileCardWrapper__cLvTI [href]")
    try:
      links = [elem.get_attribute('href') for elem in elems]
    except exceptions.StaleElementReferenceException:
      continue
    all_links.append(links)
  #Flatten the list of lists into a single order list
  flat_links = [link for all_links in all_links for link in all_links]

  #Differentiate between creators and influencers
  creator_links = []
  influencer_links = []

  for link in flat_links:
    creator_show = re.findall('https://www.snapchat.com/p/.*', link)
    creator_links.append(creator_show)

    influ_shows = re.findall('https://www.snapchat.com/add/.*', link)
    influencer_links.append(influ_shows)

  #Flatten lists
  flat_creator = [creator for creator_links in creator_links for creator in creator_links]
  flat_influencer = [infl for influencer_links in influencer_links for infl in influencer_links]

  #Finalize the list of creators by removing any duplicate entries
  creator_noduplicates = list(OrderedDict.fromkeys(flat_creator))

  return creator_noduplicates


#Metric Scrape Function
def full_scrape():
  #Configure browser
  browser = driver_no_proxy()

  #Call proxy identification function to get ip address
  ip = proxy_identification()
  browser.delete_all_cookies()

  #Getting IP location information
  #Go to ip lookup webiste
  ip_lookup = 'https://whatismyipaddress.com/ip-lookup'
  browser.get(ip_lookup)
  #Find the ip input box, write in the ip address and search results
  input_element = browser.find_element(By.XPATH, '//*[@id="fl-post-162"]/div[1]/div[1]/div[1]/div/div[2]/div/div/div/div/div[1]/div/div[2]/div/div/form/label/input')
  input_element.send_keys(ip)
  input_element.send_keys(Keys.RETURN)

  time.sleep(5)

  #Store country and city location of ip address from title of browser page
  address_lookup = browser.title
  country_list = re.findall('\,\s(.*)', address_lookup)
  country = "".join(country_list)
  city_list = re.findall('in\s(.+?),', address_lookup)
  city = "".join(city_list)

  #Create lists to store data from scrape which will eventually represent different table fields
  page_list = []
  type_list = []
  source_list = []
  ip_country = []
  ip_city = []
  channel_list = []
  subs_list = []
  descrp_list = []
  num_eps_list = []
  landing_ep_list = []
  landing_info_list = []
  landing_thumb_list = []
  #Final link list (for waiting)
  final_links = []
  #Timestamp
  time_list = []

  #Metrics Scrape
  creator_list = sfy_scrape()
  #Visiting each profile in the creator list and scraping the desired metrics
  for creator in creator_list:
    browser.get(creator)

    #Get metrics of interest
    webpage = browser.title
    page_list.append(webpage)

    distro_type = 'SFY'
    type_list.append(distro_type)

    content_source = 'Creator Show'
    source_list.append(content_source)

    #Append ip country and city from above steps
    ip_country.append(country)
    ip_city.append(city)

    #Get channel name
    try:
      channel = browser.find_element(By.CLASS_NAME, "PublicProfileDetailsCard_inlineDiv__V12Dg").text
      channel_list.append(channel)
    except NoSuchElementException:
      channel=np.nan
      subs=np.nan
      description = np.nan
      num_eps = np.nan
      landing_episode = np.nan
      landing_info = np.nan
      landing_thumbnail = np.nan

      channel_list.append(channel)
      subs_list.append(subs)
      descrp_list.append(description)
      num_eps_list.append(num_eps)
      landing_ep_list.append(landing_episode)
      landing_info_list.append(landing_info)
      landing_thumb_list.append(landing_thumbnail)
      final_links.append(np.nan)
      time_list.append(np.nan)

      continue

    #Get subscriber base
    try:
      subs = browser.find_element(By.CLASS_NAME, "PublicProfileDetailsCard_desktopSubscriberTextOnMedia__l0rjj").text
    except NoSuchElementException:
      subs=np.nan
    subs_list.append(subs)

    #Get channel description
    description = browser.find_element(By.CLASS_NAME, "PublicProfileCard_desktopTitle__9ik6D").text
    descrp_list.append(description)

    #Get the most recent episodes location, and respective metrics
    #Scroll to bottom of page
    SCROLL_PAUSE_TIME = 5
    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    episodes = browser.find_elements(By.CLASS_NAME, 'StoryListTile_title__uu0Lo')
    num_eps = len(episodes)
    num_eps_list.append(num_eps)

    if num_eps > 0:
      landing_episode = episodes[(num_eps-1)].text

      all_episodes_info = browser.find_elements(By.CLASS_NAME, 'StoryListTile_storyInfo__XnOTC')
      landing_info = all_episodes_info[(num_eps-1)].text

      multiple = browser.find_elements(By.CSS_SELECTOR, ".StoryListTile_thumbnail__NYD_G [src]")
      landing_thumbnail = multiple[(num_eps-1)].get_attribute('src')

    else:
      landing_episode = np.nan
      landing_info = np.nan
      landing_thumbnail = np.nan

    #Append all recent episode metrics to lists
    landing_ep_list.append(landing_episode)
    landing_info_list.append(landing_info)
    landing_thumb_list.append(landing_thumbnail)


    #Append link to final link list
    final_links.append(creator)

    #Timestamp
    utc = pd.Timestamp.today().floor('MIN').to_numpy()
    timestamp = utc - np.timedelta64(4, 'h')
    time_list.append(timestamp)

    browser.delete_all_cookies()
    time.sleep(10)

  #Make final adjustments and create table from scraped data

  #Create "distribution ranking" field ranking each channel from when it showed up in the scrape
  list_ranking = []
  ranking = 0
  for channel in range(len(channel_list)):
    ranking += 1
    list_ranking.append(ranking)

  #Create dataframe
  final_df = pd.DataFrame({'Page Summary': page_list,
                         'Distro Type': type_list,
                         'Content Source': source_list,
                         'Channel/Show': channel_list,
                         "Link": final_links,
                         "Subscriber Base": subs_list,
                         'Description': descrp_list,
                         'Landing Episode': landing_ep_list,
                         "Landing Info": landing_info_list,
                         "# of Episodes": num_eps_list,
                         "Timestamp": time_list,
                         "Landing Episode Thumb": landing_thumb_list,
                         'Distribution Ranking': list_ranking,
                         'ip_address_country': ip_country,
                         'ip_address_city': ip_city
                         })

  #Adjust datatypes
  final_df['Timestamp'] = final_df['Timestamp'].astype('str')

  #Clean subscribers column and convert dtype to integer
  final_df['subsciber_base_clean'] = final_df['Subscriber Base'].str.replace('(?<=\.[0-9])k.*|(?<=\.[0-9])0k.*', '00')
  final_df['subsciber_base_clean'] = final_df['subsciber_base_clean'].str.replace('(?<!\.\d)k.*', '000')
  final_df['subsciber_base_clean'] = final_df['subsciber_base_clean'].str.replace('(?<=\.[0-9])m.*|(?<=\.[0-9])[0-9]m.*', '00000')
  final_df['subsciber_base_clean'] = final_df['subsciber_base_clean'].str.replace('\.', '')
  final_df['subsciber_base_clean'] = final_df['subsciber_base_clean'].fillna(0)
  final_df['subsciber_base_clean'] = final_df['subsciber_base_clean'].astype('int')

  #Clean recent episode information and store in seperate data fields
  final_df['landing_published_date'] = final_df['Landing Info'].str.extract('^(.+?)•')
  final_df['series_number'] = final_df['Landing Info'].str.extract('\•(.+?),')
  final_df['episode_number'] = final_df['Landing Info'].str.extract('\,(.*)')

  #Fill null values with 0
  #final_df = final_df.fillna(0)

  return final_df


# **Call Functions**

In [ ]:
#Call sfy_feed() function  i.e just scrape the channels in the SFY feed and store their urls in a list
sfy_list = sfy_scrape()

In [ ]:
sfy_list

['https://www.snapchat.com/p/960eaf36-26ff-4ee9-8075-93e5e7f25d62',
 'https://www.snapchat.com/p/13818e39-b672-4c9f-90e5-bb41b251e8de',
 'https://www.snapchat.com/p/d8d55145-93ef-4767-8f78-e582558aabe4',
 'https://www.snapchat.com/p/85e1bfa3-259b-479c-ac4f-0921b37ca471',
 'https://www.snapchat.com/p/0cfdeaba-6c0c-4f27-b36d-9ad2d45e19f8',
 'https://www.snapchat.com/p/ba2f1ba2-ee93-491d-b344-cc16e88e28aa',
 'https://www.snapchat.com/p/f770f9fb-2b66-4ca2-8d17-65116ef2101a',
 'https://www.snapchat.com/p/ec1ea6af-9a90-49c1-a109-a77a37d36c99',
 'https://www.snapchat.com/p/9decdcd3-d5b8-48c8-870e-6a6b076c7d9d',
 'https://www.snapchat.com/p/f243ef47-e4a9-48fc-80b6-f8c3778d6b3f',
 'https://www.snapchat.com/p/dfc593e8-4cae-40ae-b002-6f67ad8ea28c',
 'https://www.snapchat.com/p/3e08a1e5-23c3-498d-9248-d8ed7d2fa282',
 'https://www.snapchat.com/p/2719ecd1-af5a-4568-9bb3-0be719df1dd7',
 'https://www.snapchat.com/p/f6fa411f-9c14-4b18-8fec-3e7775dc81df',
 'https://www.snapchat.com/p/e0c51e75-bb5b-4c1a-

In [ ]:
#Call full scrape() function - i.e scrape the list of channels and scrape all relevant metrics associated with each channel
full_df = full_scrape()

<ipython-input-6-11323b83b175>:272: FutureWarning: The default value of regex will change from True to False in a future version.
  final_df['subsciber_base_clean'] = final_df['Subscriber Base'].str.replace('(?<=\.[0-9])k.*|(?<=\.[0-9])0k.*', '00')
<ipython-input-6-11323b83b175>:273: FutureWarning: The default value of regex will change from True to False in a future version.
  final_df['subsciber_base_clean'] = final_df['subsciber_base_clean'].str.replace('(?<!\.\d)k.*', '000')
<ipython-input-6-11323b83b175>:274: FutureWarning: The default value of regex will change from True to False in a future version.
  final_df['subsciber_base_clean'] = final_df['subsciber_base_clean'].str.replace('(?<=\.[0-9])m.*|(?<=\.[0-9])[0-9]m.*', '00000')
<ipython-input-6-11323b83b175>:275: FutureWarning: The default value of regex will change from True to False in a future version.
  final_df['subsciber_base_clean'] = final_df['subsciber_base_clean'].str.replace('\.', '')


In [10]:
full_df

,Page Summary,Distro Type,Content Source,Channel/Show,Link,Subscriber Base,Description,Landing Episode,Landing Info,# of Episodes,Timestamp,Landing Episode Thumb,Distribution Ranking,ip_address_country,ip_address_city,subsciber_base_clean,landing_published_date,series_number,episode_number
0,BigHeadJustice | Snapchat,SFY,Creator Show,BigHeadJustice,https://www.snapchat.com/p/13818e39-b672-4c9f-90e5-bb41b251e8de,26.5k Subscribers,"From tutorials and hair care to edgy looks, afro artist BigHeadJustice (2.5M+ social followers) makes natural hair videos to help you grow out your locks!",She made her hair 3x bigger 😳,"Jul 7•Series 1, Episode 1",1.0,2023-07-14 15:29:00,"https://cf-st.sc-cdn.net/aps/bolt_web/aHR0cHM6Ly9jZi1zdC5zYy1jZG4ubmV0L2Fwcy9ib2x0X3N0YWdnL2FIUjBjSE02THk5alppMXpkQzV6WXkxalpHNHVibVYwTDJRdmFrTldaRlZqWmt4eGMyMTRjREpTVVhsYVNrdGxMak14TUQ5dGJ6MUhhMFZoUkVKdlFVZG5RWGxCWjFJNVZVRldaMEZXYjFsVlNFcHNZbGRzTVdKV1RqQmlNMG8xVmtkc2MxcFdXbWhqYld4b1ltNVNlbTluUlV4RFRGbERTV2RaUzBGclowTkZaME5wUVZGblNVRlRTVVZEWjBwSlFWRWxNMFFsTTBRbWRXTTlOUS5SUzU0MCww._RS48,72_FMwebp",1,Netherlands,Groningen,26500,Jul 7,Series 1,Episode 1
1,Vision | Snapchat,SFY,Creator Show,Vision,https://www.snapchat.com/p/85e1bfa3-259b-479c-ac4f-0921b37ca471,1.77m Subscribers,Covering the world of technology and innovation.,Oddly Satisfying Inventions!,"Thu•Series 3, Episode 31",30.0,2023-07-14 15:29:00,"https://cf-st.sc-cdn.net/aps/bolt_web/aHR0cHM6Ly9jZi1zdC5zYy1jZG4ubmV0L2Fwcy9ib2x0X3N0YWdnL2FIUjBjSE02THk5alppMXpkQzV6WXkxalpHNHVibVYwTDJRdmNHWTNabGx1UlZWRFZVTm9Ra0ZLVVhSQk4wZ3dMak14TUQ5dGJ6MUhhMFZoUkVKdlFVZG5RWGxCWjFJNVZVRldaMEZXYjFsVlNFcHNZbGRzTVdKV1RqQmlNMG8xVmtkc2MxcFdXbWhqYld4b1ltNVNlbTluUlV4RFRGbERTV2RaUzBGclowTkZaME5wUVZGblNVRlRTVVZEWjBwSlFWRWxNMFFsTTBRbWRXTTlOUS5SUzU0MCww._RS48,72_FMwebp",2,Netherlands,Groningen,1700000,Thu,Series 3,Episode 31
2,Hasbulla Driving Skills 😱 - Car World | Snapchat,SFY,Creator Show,Car World,https://www.snapchat.com/p/d8d55145-93ef-4767-8f78-e582558aabe4,88.3k Subscribers,Dive into the car world and get the latest news and trends in the automotive world hosted by Dano!,$5 Million Batmobile 🤯,"6h ago•Series 1, Episode 66",28.0,2023-07-14 15:30:00,"https://cf-st.sc-cdn.net/aps/bolt_web/aHR0cHM6Ly9jZi1zdC5zYy1jZG4ubmV0L2Fwcy9ib2x0X3N0YWdnL2FIUjBjSE02THk5alppMXpkQzV6WXkxalpHNHVibVYwTDJRdmNISmxWalp5Ykd4V2QwZGtja2gxVFdSVU5tbHVMak14TUQ5dGJ6MUhhMFZoUkVKdlFVZG5RWGxCWjFJNVZVRldaMEZXYjFsVlNFcHNZbGRzTVdKV1RqQmlNMG8xVmtkc2MxcFdXbWhqYld4b1ltNVNlbTluUlV4RFRGbERTV2RaUzBGclowTkZaME5wUVZGblNVRlRTVVZEWjBwSlFWRWxNMFFsTTBRbWRXTTlOUS5SUzU0MCww._RS48,72_FMwebp",3,Netherlands,Groningen,88300,6h ago,Series 1,Episode 66
3,Dads | Snapchat,SFY,Creator Show,Dads,https://www.snapchat.com/p/960eaf36-26ff-4ee9-8075-93e5e7f25d62,615k Subscribers,Discover the most viral clips of fathers indulging in entertaining and hilarious activities!,Pushing dad’s limits,"7h ago•Series 1, Episode 41",4.0,2023-07-14 15:30:00,"https://cf-st.sc-cdn.net/aps/bolt_web/aHR0cHM6Ly9jZi1zdC5zYy1jZG4ubmV0L2Fwcy9ib2x0X3N0YWdnL2FIUjBjSE02THk5alppMXpkQzV6WXkxalpHNHVibVYwTDJRdlozWk5XazFFVVVGak1XbDJWVXA1VDNSWFRXaHVMak14TUQ5dGJ6MUhhMFZoUkVKdlFVZG5RWGxCWjFJNVZVRldaMEZXYjFsVlNFcHNZbGRzTVdKV1RqQmlNMG8xVmtkc2MxcFdXbWhqYld4b1ltNVNlbTluUlV4RFRGbERTV2RaUzBGclowTkZaME5wUVZGblNVRlRTVVZEWjBwSlFWRWxNMFFsTTBRbWRXTTlOUS5SUzU0MCww._RS48,72_FMwebp",4,Netherlands,Groningen,615000,7h ago,Series 1,Episode 41
4,Science Is Cool | Snapchat,SFY,Creator Show,Science Is Cool,https://www.snapchat.com/p/b6e7d9e4-f666-4f99-8643-54875183a27e,65.4k Subscribers,"The Science of Satisfaction, Explained.",I Tried a New Style,"3h ago•Series 2, Episode 39",20.0,2023-07-14 15:31:00,"https://cf-st.sc-cdn.net/aps/bolt_web/aHR0cHM6Ly9jZi1zdC5zYy1jZG4ubmV0L2Fwcy9ib2x0X3N0YWdnL2FIUjBjSE02THk5alppMXpkQzV6WXkxalpHNHVibVYwTDJRdmNWbHNSV3R1VGxoM2NqRlBORmREUWxCS2MzTkRMak14TUQ5dGJ6MUhhMFZoUkVKdlFVZG5RWGxCWjFJNVZVRldaMEZXYjFsVlNFcHNZbGRzTVdKV1RqQmlNMG8xVmtkc2MxcFdXbWhqYld4b1ltNVNlbTluUlV4RFRGbERTV2RaUzBGclowTkZaME5wUVZGblNVRlRTVVZEWjBwSlFWRWxN

# **Export to G-sheet**

In [11]:
#Export to Google Sheets / Part 1 Auth
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

In [29]:
gc = gspread.authorize(creds)
worksheet = gc.open('Snapchat Scraping Database').worksheet('New_Channel_Scrape')

In [30]:
worksheet.update('A1:S38', [full_df.columns.values.tolist()] + full_df.values.tolist())

{'spreadsheetId': '1Up2QwOY0R9FlJsNkzOZOh5-eHH6pC7JeHaJYyClATUo',
 'updatedRange': 'New_Channel_Scrape!A1:S38',
 'updatedRows': 38,
 'updatedColumns': 19,
 'updatedCells': 722}